# Parallel Computing with IPython Parallel 

In regular IPython we have a `client` (the frontend) and a `kernel` which executes the code. And they communciate with messages. 

So, as IPython already does remote execution... if you have _one_ remote kernel, why not have _multiple_?

This is the basis of IPython Parallel (or ipyparallel).

One or many IPython Parallel "engines" execute code, and they are managed by the IPython "controller". 

<div>
<img src="./ipyparallel.png" style="width:300px"/>
</div>
Rather than having clients (blue) connect directly to kernels (green) as in notebook, you have an intermediary of a hub (with schedulers) - known as the "controller". The client communicates only with the controller. The controller keeps track of the available engines and forwards requests from the client to the engines. It schedules the work and monitors its status. The results are communicated through the controller back to the client.

To use IPython for parallel computing, you need to start one instance of the controller and one or more instances of the engines. The controller and each engine can run on different machines or on the same machine.

There are three ways to start the controller and engines:

1. Separately, using the **ipcontroller** and **ipengine** commands.
2. In a more automated way using the **ipcluster** command.
3. From a custom **magic** developed inhouse `import ipcmagic`

Right now we will try the first method, which is more "manual", but provides the most transparency.

<div class="alert alert-block alert-info">
    <b>Note:</b> The following commands need to be entered in a terminal. File > New > Terminal. A terminal will open as a new tab. Grab the tab and pull it to the right to have the terminal next to your notebook.
</div>

```
$ ipcontroller start &
$ srun -n 4 ipengine --mpi 
```    

The IPython Parallel engines need to be started using the `mpirun` command (or equivalent). On our system:

- Start the **ipcontroller** 
- Start the **ipengines** (using `srun` and with `--mpi` argument if you want to use MPI).


Now let's see how we access our "Cluster". [IPython][IP] comes with a module [ipyparallel][IPp] that is used to access the engines, we just started. We first need to import Client.

[IPp]: https://ipyparallel.readthedocs.io/en/latest/
[IP]: http://www.ipython.org

The client is started by first importing it from ipyparallel and then by initalizing it. 

In [1]:
import ipyparallel as ipp

<div class="alert alert-block alert-danger">
    <b>Note:</b> If you receive an error ModuleNotFoundError: No module named 'ipyparallel', ensure that you have the miniconda-ss2020 kernel loaded
</div>


In [2]:
rc = ipp.Client(profile="default")

List the ids of the engines attached:

In [5]:
rc.ids

[0, 1, 2, 3]

## Parallel Magics

IPython makes it very easy to use IPyParallel. It provides the magic commands ``%px`` and ``%%px`` to execute code in parallel. The target attribute is used to pick the engines, you want. By default, all the engines of the last Client object created are used. 

In [8]:
%%px --target 0:5
import os, socket

print(os.getpid())
print(socket.gethostname())

[stdout:0] 
30966
nid03887
[stdout:1] 
30967
nid03887
[stdout:2] 
30968
nid03887
[stdout:3] 
30969
nid03887


### You can do even do MPI

In [9]:
%%px
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = MPI.COMM_WORLD.Get_rank()
size = MPI.COMM_WORLD.Get_size()

A = np.zeros((size,size))
if rank==0:
    A = np.random.randn(size, size)
    print("Original array on root process\n", A)

local_a = np.zeros(size)
comm.Scatter(A, local_a, root=0)
print("\nProcess", rank, "received", local_a)

[stdout:0] 
Original array on root process
 [[ 2.67511135  1.48369085  1.38100283  0.82825596]
 [ 0.3602964   1.16803423 -1.22454897 -0.65310519]
 [-0.65298612  0.5351841   2.42297397  2.08926872]
 [ 0.19005168 -0.74049525  0.54114466 -0.08386228]]

Process 0 received [2.67511135 1.48369085 1.38100283 0.82825596]
[stdout:1] 
Process 1 received [ 0.3602964   1.16803423 -1.22454897 -0.65310519]
[stdout:2] 
Process 2 received [-0.65298612  0.5351841   2.42297397  2.08926872]
[stdout:3] 
Process 3 received [ 0.19005168 -0.74049525  0.54114466 -0.08386228]


### Autopx magic

In [12]:
print("Process", rank, "received", local_a)

[stdout:0] Process 0 received [2.67511135 1.48369085 1.38100283 0.82825596]
[stdout:1] Process 1 received [ 0.3602964   1.16803423 -1.22454897 -0.65310519]
[stdout:2] Process 2 received [-0.65298612  0.5351841   2.42297397  2.08926872]
[stdout:3] Process 3 received [ 0.19005168 -0.74049525  0.54114466 -0.08386228]


In [11]:
%autopx

%autopx enabled


## IPCMagic

As mentioned earlier you can start and stop an ipyparallel cluster with a magic command. To make it availalbe import `ipcmagic`.

In [ ]:
import ipcmagic
import ipyparallel as ipp

In [14]:
%ipcluster start -n 2 --mpi

KeyboardInterrupt: 

In [3]:
rc = ipp.Client()
rc.ids

[0, 1]

In [4]:
%%px
import os, socket
print(socket.gethostname())

[stdout:0] nid03898
[stdout:1] nid03898


In [5]:
%ipcluster stop 